<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP_2022/blob/main/04%20Deep%20Learning/notebooks/04-Herramientas-Adicionales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Herramientas Adicionales</h1>

El objetivo de esta notebook es mostrar algunas herramientas adicionales para mejorar el entrenamiento y/o desempeño de redes neuronales. En particular, veremos:

* Callbacks
    * Early Stopping
    * Checkpoint
    * Reduce on Plateu
* Dropout
* Batch Normalization
* Gridsearch

Además, realizaremos ejemplos de clasificación binaria.

Recuerda la simbología de las secciones:

* 🔽 Esta sección no forma parte del proceso usual de Machine Learning. Es una exploración didáctica de algún aspecto del funcionamiento del algoritmo.
* ⚡ Esta sección incluye técnicas más avanzadas destinadas a optimizar o profundizar en el uso de los algoritmos.
* ⭕ Esta sección contiene un ejercicio o práctica a realizar. Aún si no se establece una fecha de entrega, es muy recomendable realizarla para practicar conceptos clave de cada tema.

⚡ De esta forma podemos verificar que tenemos una GPU:

In [ ]:
import tensorflow as tf

print('GPU presente en: {}'.format(tf.test.gpu_device_name()))

# Callbacks

Un *callback* es un objeto que puede realizar acciones en varias etapas del entrenamiento (por ejemplo, al inicio o al final de una época, antes o después de un *batch*, etc.).

Puedes usar *callbacks* para:

* Escribir los registros de TensorBoard después de cada lote de entrenamiento para monitorizar tus métricas
* Guardar periódicamente tu modelo en el disco
* Hacer un *early stopping*.
* Obtener una visión de los estados internos y las estadísticas de un modelo durante el entrenamiento.

Podemos consultar la lista completa de callbacks en https://keras.io/api/callbacks/

Para ilustrar algunos callbacks, y otro tipo de capas, consideremos el siguiente ejemplo. Entrenaremos una red neuronal MLP para la tarea de **clasificación multiclase** en el siguiente dataset Fashion MNIST.

In [ ]:
from keras.datasets import fashion_mnist

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

| Label | Clase               |
|-------|---------------------|
| 0     | T-shirt/top         |
| 1     | Trouser             |
| 2     | Pullover            |
| 3     | Dress               |
| 4     | Coat                |
| 5     | Sandal              |
| 6     | Shirt               |
| 7     | Sneaker             |
| 8     | Bag                 |
| 9     | Ankle boot          |

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

idxs = np.random.choice(X_train.shape[0],5,replace=False)

fig, axs = plt.subplots(nrows=1, ncols=5,figsize=(12,6))
for idx, ax in zip(idxs, axs.flatten()):
    ax.imshow(X_train[idx], cmap='gray')
    ax.set_title(f"Label: {y_train[idx]}")
    ax.axis('off')
fig.show()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,train_size=0.8,random_state=199)

In [ ]:
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

labels, conteos = np.unique(y_train,return_counts=True)

plt.figure()
plt.bar(labels,conteos)
plt.show()

In [ ]:
from keras.utils import to_categorical

y_train_cat = to_categorical(y_train)
y_val_cat = to_categorical(y_val)
y_test_cat = to_categorical(y_test)

print(f"y_train_cat shape: {y_train_cat.shape}")
print(f"y_val_cat shape: {y_val_cat.shape}")
print(f"y_test_cat shape: {y_test_cat.shape}")

Definamos una función para crear el modelo base con el que estaremos experimentando

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Input, Flatten


def build_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Flatten())
    model.add(Dense(500, activation='relu'))

    #---- Completa la capa de salida -----
    model.add(Dense(10, activation='softmax'))
    #-------------------------------------
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

## ⚡ Callbacks: `EarlyStopping`

El [`EarlyStopping`](https://keras.io/api/callbacks/early_stopping/) es un *callback* que nos permite detener el entrenamiento para evitar el overfitting. El callback monitorea el entrenamiento y lo detiene en el momento en el que una métrica (o perdida) deja de mejorar.

Algunos de los principales hiperparámetros son:

* `monitor`: Cantidad a controlar.
* `min_delta`: Cambio mínimo en la cantidad monitorizada para calificar como mejora, es decir, un cambio absoluto inferior a min_delta, contará como no mejora.
* `patience`: Número de épocas sin mejora tras las cuales se detendrá el entrenamiento.
* `mode`: Puede ser {"auto", "min", "max"}. En el modo "min", el entrenamiento se detendrá cuando la cantidad supervisada haya dejado de disminuir; en el modo "max", se detendrá cuando la cantidad supervisada haya dejado de aumentar; en el modo "auto", la dirección se deduce automáticamente del nombre de la cantidad o métrica supervisada.
* `restore_best_weights`: Indica si se restauran los pesos del modelo a la época con el mejor valor en la cantidad monitoreada.

Definimos y entrenamos una red neuronal. Observa el número de neuronas en la capa de salida y la función de activación.

Su desempeño tendrá un accuracy mucho menor en el conjunto de prueba. Observa las curvas de aprendizaje, **es un caso claro de overfitting**.

*El tiempo de ejecución es de alrededor de 4 minutos*

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Input, Flatten

input_shape = X_train[0].shape # Extraemos el shape de cualquiera de los
print(f"Input shape: {input_shape}")

model = build_model(input_shape)

#----- Entremamos el modelo ------
history = model.fit(X_train, y_train_cat,
                    validation_data=(X_val, y_val_cat),
                    epochs=35
                    )

In [ ]:
# evaluamos el modelo
_, train_acc = model.evaluate(X_train, y_train_cat, verbose=0) # No nos importa guardar el loss en una variable
_, val_acc = model.evaluate(X_val, y_val_cat, verbose=0)
print(f'Train accuracy: {round(train_acc,3)}. Validation accuracy : {round(val_acc,3)}')

# ---- graficamos la función de perdida ----
plt.figure(figsize=(11,5))
plt.subplot(1,2,1)
plt.suptitle("Validation and Training Loss",fontsize=14)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
# ---- graficamos la métrica de rendimiento ----
plt.subplot(1,2,2)
plt.suptitle("Validation and Training Accuracy",fontsize=14)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend()
plt.show()

Ahora, usemos el callback para detener el entrenamiento en el momento adecuado.

Definimos el *callback*. Es una clase por lo que tenemos que inicializarla con hiperparámetros y obtenemos un objeto.

In [ ]:
from keras.callbacks import EarlyStopping

callback_es = EarlyStopping(monitor='val_loss',
                   mode='min',
                   verbose=1,
                   patience=3,
                   restore_best_weights=True
                   )

Entrenamos usando el *callback*. Observa en cuántas épocas detiene el entrenamiento.

In [ ]:
model = build_model(input_shape)

history = model.fit(X_train, y_train_cat,
                    validation_data=(X_val, y_val_cat),
                    epochs=35,
                    # verbose=0,
                    callbacks=[callback_es]
                    )


_, train_acc = model.evaluate(X_train, y_train_cat, verbose=0) # Observar qué está pasando aquí
_, val_acc = model.evaluate(X_val, y_val_cat, verbose=0)
print(f"Train accuracy: {train_acc}.\nValidation accuracy: {val_acc}")

🔵 Observa que el accuracy en la validación mejoró y en el entrenamiento empeoró un poco, ¿qué interpretación le damos a esto?

Graficamos el entrenamiento

In [ ]:
# ---- graficamos la función de perdida ----
plt.figure(figsize=(11,5))
plt.subplot(1,2,1)
plt.suptitle("Validation and Training Loss",fontsize=14)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
# ---- graficamos la métrica de rendimiento ----
plt.subplot(1,2,2)
plt.suptitle("Validation and Training Accuracy",fontsize=14)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend()
plt.show()

## ⚡ Callbacks: `ModelCheckpoint`

Este callback sirve para guardar el modelo en el momento en que comenzó el overfitting y se comenzó a perder accuracy en el conjunto de validación. Algunos parámetros importantes:

* `save_best_only`: if save_best_only=True, it only saves when the model is considered the "best" and the latest best model according to the quantity monitored will not be overwritten. If filepath doesn't contain formatting options like {epoch} then filepath will be overwritten by each new better model.
* `mode`: one of {'auto', 'min', 'max'}. If save_best_only=True, the decision to overwrite the current save file is made based on either the maximization or the minimization of the monitored quantity. For val_acc, this should be max, for val_loss this should be min, etc. In auto mode, the mode is set to max if the quantities monitored are 'acc' or start with 'fmeasure' and are set to min for the rest of the quantities.
* `save_weights_only`: if True, then only the model's weights will be saved (model.save_weights(filepath)), else the full model is saved (model.save(filepath)).

Definimos el modelo

In [ ]:
model = build_model(input_shape)

Creamos el callback

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath = 'best_model.keras'
callback_cp_best = ModelCheckpoint(
                            filepath=filepath,
                            monitor='val_loss',
                            verbose=1,
                            save_best_only=True,
                            mode='min'
                            )

callbacks = [callback_cp_best]

También podemos combinar callbacks

In [ ]:
# callbacks = [callback_cp_best,callback_es]

⚡**Por ahora no lo ejecutemos.** También podemos guardar varios modelos, con información sobre la época y loss

In [ ]:
# filepath = 'my_best_model.epoch{epoch:02d}-loss{val_loss:.2f}.keras'

# callback_cp_all = ModelCheckpoint(filepath=filepath,
#                              monitor='val_loss',
#                              verbose=1,
#                              save_best_only=True,
#                              mode='min')
# callbacks = [callback_cp_all]

Entrenamos el modelo usando el callback definido previamente. Observar que, en este caso, realizará el entrenamiento con todas las épocas y sólo guardará el módelo cuando alcance un nuevo mínimo en la perdida de la validación.

In [ ]:
history = model.fit(X_train, y_train_cat,
                    validation_data=(X_val, y_val_cat),
                    epochs=25,
                    callbacks=callbacks)

In [ ]:
# ---- graficamos la función de perdida ----
plt.figure(figsize=(11,5))
plt.subplot(1,2,1)
plt.suptitle("Validation and Training Loss",fontsize=14)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
# ---- graficamos la métrica de rendimiento ----
plt.subplot(1,2,2)
plt.suptitle("Validation and Training Accuracy",fontsize=14)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend()
plt.show()

### ⚡ Leemos el modelo y realizamos predicciones con él

Leemos y evaluamos usando el modelo guardado del callback anterior.

**En esta parte, además, evaluamos *externamente* las métricas usuales de rendimiento.**

In [ ]:
from keras.models import load_model

filepath = '/content/best_model.keras'

model_reloaded = load_model(filepath)
y_pred_cat = model_reloaded.predict(X_val)

Observa la forma que tienen las predicciones, son probabilidades de pertenecer a la clase positiva (la clase 1). Recuerda que la última capa tiene una activación sigmoide que está en un rango $(0,1)$.

In [ ]:
print(y_pred_cat.shape)
np.round(y_pred_cat[:5],3)

Convirtamoslas a predicciones de clases para fin de evaluar también usando las métricas de rendimiento de clasificación de scikit-learn (precision, accuracy, etc).

In [ ]:
import numpy as np

y_pred = np.argmax(y_pred_cat,axis=1)
print(y_pred.shape)
y_pred[:5]

Ahora sí, podemos evaluar. Recordar que, para el `roc_auc_score` necesitamos las probabilidades de pertenecer a la clase.

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

print(f"Accuracy: {accuracy_score(y_val,y_pred)}")
print(f"F1 Score: {f1_score(y_val,y_pred,average='macro')}")
print(f"ROC-AUC Score: {roc_auc_score(y_val,y_pred_cat,multi_class='ovr')}")

# Dropout

Es fácil que las redes neuronales de aprendizaje profundo se sobreajusten rápidamente a un conjunto de datos de entrenamiento con pocos ejemplos.

Se sabe que los conjuntos de redes neuronales con diferentes configuraciones de modelos reducen el sobreajuste, pero requieren el gasto computacional adicional de entrenar y mantener múltiples modelos.

Se puede utilizar un único modelo para simular que se dispone de un gran número de arquitecturas de red diferentes mediante la eliminación aleatoria de nodos durante el entrenamiento. Esto se denomina *dropout* y ofrece un método de regularización muy barato desde el punto de vista computacional y notablemente eficaz para reducir el sobreajuste y mejorar el error de generalización en redes neuronales profundas de todo tipo.

**Esta estrategia no siempre mejora el rendimiento de la red y hay opiniones divididas en cuanto a su eficacia. Sin embargo, es una técnica clásica del deep learning.**


<img align="center" width="50%" src="https://github.com/DCDPUAEM/DCDP/blob/main/04%20Deep%20Learning/img/dropout.png?raw=1"/>

[Artículo donde se propuso](https://arxiv.org/abs/1207.0580)


----

El Dropout es como estudiar en grupo vs. estudiar siempre con los mismos amigos


* Sin Dropout (Overfitting): Imagina que siempre estudias con el mismo grupo de 3 amigos muy inteligentes. Te acostumbras tanto a sus formas de explicar y resolver problemas que:

 * En casa, con ellos, resuelves todo perfectamente
 * Pero en el examen real (solo), te bloqueas porque no están tus amigos para ayudarte
 * Te volviste demasiado dependiente de ellos

* Con Dropout: Ahora imagina que cada día de estudio, aleatoriamente algunos de tus amigos no pueden venir:

 * Te vuelves más independiente y versátil
 * Aprendes a resolver problemas sin depender de personas específicas
 * En el examen real, rindes mejor porque no dependes de nadie más
 * Desarrollas tus propias estrategias

## Sin dropout

In [ ]:
from keras.layers import Dropout, Input, Flatten, Dense
from keras.models import Sequential

def build_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    #---- Completa la capa de salida -----
    model.add(Dense(10, activation='softmax'))
    #-------------------------------------
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
input_shape = X_train[0].shape # Extraemos el shape de cualquiera de los
print(f"Input shape: {input_shape}")

model = build_model(input_shape)

history = model.fit(X_train, y_train_cat,
                    validation_data=(X_val, y_val_cat),
                    epochs=30,
                    # verbose=0
                    )

In [ ]:
import matplotlib.pyplot as plt

# evaluate the model
_, train_acc = model.evaluate(X_train, y_train_cat)
_, test_acc = model.evaluate(X_test, y_test_cat)
print(f"Train accuracy: {round(train_acc,3)}\nTest accuracy : {round(test_acc,3)}")

# ---- graficamos la función de perdida ----
plt.figure(figsize=(11,5))
plt.subplot(1,2,1)
plt.suptitle("Validation and Training Loss",fontsize=14)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
# ---- graficamos la métrica de rendimiento ----
plt.subplot(1,2,2)
plt.suptitle("Validation and Training Accuracy",fontsize=14)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend()
plt.show()

## Usando dropout: Efecto en el overfitting

Usaremos la misma arquitectura general de la red. Añadimos dos capas de dropout, las tasas de dropout fueron seleccionadas con gridsearch.

In [ ]:
from keras.layers import Dropout, Input, Flatten, Dense
from keras.models import Sequential

def build_model_droput(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))  # Dropout más agresivo
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.4))  # Dropout en cada capa densa
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    #---- Completa la capa de salida -----
    model.add(Dense(10, activation='softmax'))
    #-------------------------------------
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
input_shape = X_train[0].shape # Extraemos el shape de cualquiera de los

model_do = build_model_droput(input_shape=input_shape)

history = model_do.fit(X_train, y_train_cat,
                        validation_data=(X_val, y_val_cat),
                        epochs=25)

In [ ]:
import matplotlib.pyplot as plt

# evaluate the model
_, train_acc = model_do.evaluate(X_train, y_train_cat)
_, test_acc = model_do.evaluate(X_test, y_test_cat)
print(f"Train accuracy: {round(train_acc,3)}\nTest accuracy : {round(test_acc,3)}")

# ---- graficamos la función de perdida ----
plt.figure(figsize=(11,5))
plt.subplot(1,2,1)
plt.suptitle("Validation and Training Loss",fontsize=14)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
# ---- graficamos la métrica de rendimiento ----
plt.subplot(1,2,2)
plt.suptitle("Validation and Training Accuracy",fontsize=14)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend()
plt.show()

# Batch Normalization

Es una técnica que normaliza las salidas de cada capa de la red, ajustándolos para que tengan media 0 y varianza 1 por cada lote de datos. Esto acelera el entrenamiento, evita gradientes inestables y reduce la necesidad de ajustes finos en la inicialización. Además, introduce parámetros aprendibles para mantener la flexibilidad del modelo.

* Velocidad de Convergencia:
 * Con BatchNorm: El modelo aprende mucho más rápido
 * Sin BatchNorm: Convergencia más lenta, especialmente en redes profundas

* Estabilidad del Entrenamiento:
 * Con BatchNorm: Curvas de loss más suaves, menos "saltos"
 * Sin BatchNorm: Loss más errático, puede tener picos y valles

* Learning Rate:
 * Con BatchNorm: Puedes usar learning rates más altos (ej: 0.01 vs 0.001)
 * Sin BatchNorm: Necesitas learning rates más conservadores

* Accuracy Final:
 * Con BatchNorm: Generalmente mejor accuracy final
 * Sin BatchNorm: Puede alcanzar resultados similares, pero tomará más tiempo

El efecto es más notorio en redes más profundas y problemas más complejos

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input, BatchNormalization
from keras.callbacks import EarlyStopping


def build_model_bn(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    #---- Completa la capa de salida -----
    model.add(Dense(10, activation='softmax'))
    #-------------------------------------
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
input_shape = X_train[43].shape # Extraemos el shape de cualquiera de los ejemplos

model_bn = build_model_bn(input_shape)

model_bn.summary()

history = model.fit(X_train, y_train_cat,
                    validation_data=(X_val, y_val_cat),
                    epochs=25)

In [ ]:
import matplotlib.pyplot as plt

# evaluate the model
_, train_acc = model_bn.evaluate(X_train, y_train_cat)
_, test_acc = model_bn.evaluate(X_test, y_test_cat)
print(f"Train accuracy: {round(train_acc,3)}\nTest accuracy : {round(test_acc,3)}")

# ---- graficamos la función de perdida ----
plt.figure(figsize=(11,5))
plt.subplot(1,2,1)
plt.suptitle("Validation and Training Loss",fontsize=14)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
# ---- graficamos la métrica de rendimiento ----
plt.subplot(1,2,2)
plt.suptitle("Validation and Training Accuracy",fontsize=14)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend()
plt.show()

# ⚡⚡ Gridsearch

A continuación se muestra cómo realizar un gridsearch para obtener los mejores parámetros de una red neuronal, es decir, los que producen las mejores métricas. Estos parámetros pueden ser el número de neuronas, la tasa de dropout, las épocas, etc.

Para poder usar el gridsearch de scikit-learn es necesario *traducir* el módelo de red neuronal a un clasificador de scikit-learn. Esto lo hacemos con la clase `KerasClassifier`.

**⚠❗Warning**: Si se especifican un gran número de parámetros en la busqueda, esta puede tardar mucho y pueden ser penalizados en el uso de GPU en Colab. Usar con cuidado.

Seguimos con el dataset de [diabetes](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database). Resumamos el preprocesamiento en una sola celda:

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

url = "https://raw.githubusercontent.com/DCDPUAEM/DCDP/main/03%20Machine%20Learning/data/diabetes.csv"

df = pd.read_csv(url)

display(df)

X = df.iloc[:,:8].values
y = df.iloc[:,8].values

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.875,random_state=89)

idxs_to_impute = [1,2,3,4,5]
imputer = SimpleImputer(missing_values=0, strategy='mean')
X_train[:,idxs_to_impute] = imputer.fit_transform(X_train[:,idxs_to_impute])
X_test[:,idxs_to_impute] = imputer.transform(X_test[:,idxs_to_impute])

scl = StandardScaler()
X_train = scl.fit_transform(X_train)
X_test = scl.transform(X_test)

Es necesario crear una función que cree el modelo, esta debe depender de los parámetros sobre los que se quiere realizar la busqueda. Es necesario crear el modelo, compilarlo y regresarlo ya compilado.

Con la finalidad de no usar muchos recursos, problemas con un modelo muy sencillo, en el cual variaremos:

* El número de neuronas en la capa oculta.
* La activación de la capa oculta.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Input

def create_model(n_neurons,activation):
	model = Sequential()
	model.add(Input(shape=(8,)))
	model.add(Dense(n_neurons, activation=activation))
	model.add(Dense(1, activation='sigmoid'))
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [ ]:
num_neurons = [5,10,15,20,30]
activations = ['relu','sigmoid','tanh']

best_acc = 0
best_model = None
best_params = None

es = EarlyStopping(monitor='val_loss',patience=2)

for n in num_neurons:
    for act in activations:
        model = create_model(n,act)
        history = model.fit(X_train, y_train,
                            callbacks = [es],
                            epochs=50,
                            verbose=0,
                            validation_data=(X_test,y_test)
                            )
        if history.history['val_accuracy'][-1] > best_acc:
            best_acc = history.history['val_accuracy'][-1]
            best_model = model
            print(f"New best model: {best_acc}")
            best_model.save('best_model.keras')
            best_params = {'n_neurons':n,'activation':act}


In [ ]:
print(best_params)

Creamos un modelo de clasificador de scikit-learn usando la API de Keras. Esta empaqueta el módelo de keras como un estimador de scikit-learn. Después podemos usar el GridSearchCV de scikit learn.

[Documentación](https://adriangb.com/scikeras/stable/quickstart.html#training-a-model)

In [ ]:
!pip install scikeras[tensorflow]

# ⭕ Práctica

En esta práctica haremos regresión multi-output usando el dataset `mo_regression.csv`.



## Trabajo en clase

In [ ]:
import pandas as pd
import numpy as np

url = "https://raw.githubusercontent.com/DCDPUAEM/DCDP/main/04%20Deep%20Learning/data/mo_regression.csv"

df = pd.read_csv(url,index_col=0)
df

🟢 Explora el rango de las variables predictoras, ¿es necesario hacer re-escalamiento?

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(10,5))
df.iloc[:,:-3].hist()
plt.tight_layout()
plt.show()

🔴 Divide en train/validation/test usando `train_size=0.8` en ambas divisiones.

In [ ]:
from sklearn.model_selection import train_test_split

X = ...
y = ...

X_train, X_test, y_train, y_test = ...
X_train, X_val, y_train, y_val = ...

🔴 Define una red neuronal MLP para modelar este problema de regresión multi-target. La red debe tener al menos 2 capas ocultas (elige la activación de las capas ocultas).

Usa como función de perdida `MSE` y como métrica `MAE`. No olvides compilar el modelo.



In [ ]:
model = ...

Define un callback de tipo `EarlyStopping`, con paciencia 3, que esté monitoreando la perdida.

In [ ]:
from keras.callbacks import EarlyStopping



Entrena durante un número de épocas $10\leq n \leq 50$.

In [ ]:
history = ...

🟢 Grafica las curvas de entrenamiento

In [ ]:
import matplotlib.pyplot as plt

# ---- graficamos la función de perdida ----
plt.figure(figsize=(11,5))
plt.subplot(1,2,1)
plt.suptitle("Validation and Training Loss",fontsize=14)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
# ---- graficamos la métrica de rendimiento ----
plt.subplot(1,2,2)
plt.suptitle("Validation and Training MAE",fontsize=14)
plt.plot(history.history['mae'], label='train')
plt.plot(history.history['val_mae'], label='validation')
plt.legend()
plt.show()

🔴 Evalua el desempeño en el conjunto de entrenamiento y prueba usando `model.evaluate`

🔵 ¿Hay señales de overfitting? ¿Qué modificaciones al modelo consideras que se pueden hacer para mejorar el rendimiento de la tarea?

## Trabajo para entregar

Realiza los siguientes pasos:

0. Divide el conjunto de datos en 80% entrenamiento y 20% prueba.
1. Define dos modelos de red neuronal MLP para resolver esta tarea. La función de perdida a usar será 'MAE'. Un módelo tendrá una arquitectura *sencilla* y el otro, una arquitectura *compleja*. Tú decide la arquitectura concreta.
2. Entrena cada uno de ellos durante 30 épocas, compara las métricas MAE. Observa las curvas de entrenamiento.
3. Repite el entrenamiento de cada uno de ellos usando los callbacks `EarlyStopping` y `ModelCheckpoint` simultaneamente. Además, en cada uno de ellos, agrega una capa de Dropout. Observa las curvas de entrenamiento.

¿Cuál de los 4 modelos lo hizo mejor?